In [ ]:
# Install necessary packages if not already installed (uncomment if needed)
!pip install numpy==1.26 scipy==1.14 meshio==5.3.5 libigl==v2.5.1 polyscope==2.2.1 ilupp==1.0.2 ipctk==1.2.0 networkx==3.3

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 49.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 71.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 41.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.6 MB/s eta 0:00:00

In [ ]:
import meshio
import numpy as np
import scipy as sp
import igl
import polyscope as ps
import polyscope.imgui as imgui
import math
import networkx as nx
import itertools
import pbatoolkit as pbat

In [ ]:
def combine(V: list, C: list):
    Vsizes = [Vi.shape[0] for Vi in V]
    Csizes = [Ci.shape[0] for Ci in C]
    Voffsets = list(itertools.accumulate(Vsizes))
    Coffsets = list(itertools.accumulate(Csizes))
    C = [C[i] + Voffsets[i] - Vsizes[i] for i in range(len(C))]
    C = np.vstack(C)
    V = np.vstack(V)
    return V, Vsizes, C, Coffsets, Csizes

def boundary_triangles(C: np.ndarray, Coffsets: list, Csizes: list):
    F = [None]*len(Csizes)
    for i in range(len(F)):
        begin = Coffsets[i] - Csizes[i]
        end = begin + Csizes[i]
        F[i] = igl.boundary_facets(C[begin:end, :])
        F[i][:, :2] = np.roll(F[i][:, :2], shift=1, axis=1)
    Fsizes = [Fi.shape[0] for Fi in F]
    F = np.vstack(F)
    return F, Fsizes

def vertex_tetrahedron_adjacency_graph(V, C):
    row = np.repeat(range(C.shape[0]), C.shape[1])
    col = C.flatten()
    data = np.zeros_like(C)
    for i in range(C.shape[1]):
        data[:, i] = i
    data = data.flatten()
    GVT = sp.sparse.coo_array((data, (row, col)), shape=(
        C.shape[0], V.shape[0])).asformat("csc")
    return GVT

def color_dict_to_array(Cdict, n):
    C = np.zeros(n)
    keys = [key for key in Cdict.keys()]
    values = [value for value in Cdict.values()]
    C[keys] = values
    return C

def partition_vertices(GVT, dbcs):
    GVV = GVT.T @ GVT
    Gprimal = nx.Graph(GVV)
    GC = nx.greedy_color(Gprimal, strategy="random_sequential")
    GC = color_dict_to_array(GC, GVT.shape[1]).astype(np.int32)
    npartitions = GC.max() + 1
    partitions = []
    for p in range(npartitions):
        vertices = np.nonzero(GC == p)[0]
        vertices = np.setdiff1d(vertices, dbcs).tolist()
        if len(vertices) > 0:
            partitions.append(vertices)
    return partitions, GC

# Set up some example parameters (replace with your own mesh inputs if necessary)

In [ ]:
input_mesh_paths = ['mesh1.obj', 'mesh2.obj']  # Example mesh files (replace with actual paths)
output_path = "."  # Example output path
rho = 1000.
Y = 1e6
nu = 0.45
translation = 0.1
percent_fixed = 0.01
fixed_axis = 2
fixed_end = "min"

# Combine the mesh data

In [ ]:
imeshes = [meshio.read(input) for input in input_mesh_paths]
V, C = [imesh.points / (imesh.points.max() - imesh.points.min()) for imesh in imeshes], [
    imesh.cells_dict["tetra"] for imesh in imeshes]
for i in range(len(V) - 1):
    extent = V[i][:, -1].max() - V[i][:, -1].min()
    offset = V[i][:, -1].max() - V[i+1][:, -1].min()
    V[i+1][:, -1] += offset + extent*translation

# Compute mass matrix and other physical quantities

In [ ]:
detJeM = pbat.fem.jacobian_determinants(mesh, quadrature_order=2)
M = pbat.fem.MassMatrix(mesh, detJeM, rho=rho,
                        dims=1, quadrature_order=2).to_matrix()
m = np.array(M.sum(axis=0)).squeeze()

# Set up gravity field and load vector

In [ ]:
detJeU = pbat.fem.jacobian_determinants(mesh, quadrature_order=1)
GNeU = pbat.fem.shape_function_gradients(mesh, quadrature_order=1)
qgf = pbat.fem.inner_product_weights(mesh, quadrature_order=1).flatten(order="F")
Qf = sp.sparse.diags_array([qgf], offsets=[0])
Nf = pbat.fem.shape_function_matrix(mesh, quadrature_order=1)
g = np.zeros(mesh.dims)
g[-1] = -9.81
fe = np.tile(rho*g[:, np.newaxis], mesh.E.shape[1])
f = fe @ Qf @ Nf
a = f / m

# Material constants

In [ ]:
Y = np.full(mesh.E.shape[1], Y)
nu = np.full(mesh.E.shape[1], nu)
mue = Y / (2*(1+nu))
lambdae = (Y*nu) / ((1+nu)*(1-2*nu))

# Boundary conditions

In [ ]:
Xmin = mesh.X.min(axis=1)
Xmax = mesh.X.max(axis=1)
extent = Xmax - Xmin
if fixed_end == "min":
    Xmax[fixed_axis] = Xmin[fixed_axis] + percent_fixed*extent[fixed_axis]
elif fixed_end == "max":
    Xmin[fixed_axis] = Xmax[fixed_axis] - percent_fixed*extent[fixed_axis]
aabb = pbat.geometry.aabb(np.vstack((Xmin, Xmax)).T)
vdbc = aabb.contained(mesh.X)
a[:, vdbc] = 0.

# Vertex Block Descent Setup

In [ ]:
Vcollision = np.unique(F)
VC = Vcollision[:, np.newaxis].T
vbd = pbat.gpu.vbd.Vbd(V.T, VC, F.T, C.T)
vbd.a = a
vbd.m = m
vbd.wg = detJeU / 6
vbd.GNe = GNeU
vbd.lame = np.vstack((mue, lambdae))
GVT = vertex_tetrahedron_adjacency_graph(V, C)
vbd.GVT = GVT.indptr, GVT.indices, GVT.data

# Partitions and coloring

In [ ]:
GVTtopology = GVT.copy()
GVTtopology.data[:] = 1
partitions, GC = partition_vertices(GVTtopology, vdbc)
vbd.partitions = partitions
thread_block_size = 64
vbd.set_gpu_block_size(thread_block_size)

# Polyscope Setup

In [ ]:
ps.set_verbosity(0)
ps.set_up_dir("z_up")
ps.set_front_dir("neg_y_front")
ps.set_ground_plane_mode("shadow_only")
ps.set_ground_plane_height_factor(0.5)
ps.set_program_name("Vertex Block Descent")
ps.init()
vm = ps.register_volume_mesh("Simulation mesh", V, C)
vm.add_scalar_quantity("Coloring", GC, defined_on="vertices", cmap="jet")
pc = ps.register_point_cloud("Dirichlet", V[vdbc, :])

# Visualization callback function

In [ ]:
def callback():
    global dt, iterations, substeps, rho_chebyshev, thread_block_size, initialization_strategy, RdetH, kD
    global animate, export, t
    global profiler

    changed, dt = imgui.InputFloat("dt", dt)
    changed, iterations = imgui.InputInt("Iterations", iterations)
    changed, substeps = imgui.InputInt("Substeps", substeps)
    changed, rho_chebyshev = imgui.InputFloat(
        "Chebyshev rho", rho_chebyshev)
    changed, kD = imgui.InputFloat(
        "Damping", kD, format="%.8f")
    changed, RdetH = imgui.InputFloat(
        "Residual det(H)", RdetH, format="%.15f")
    changed, thread_block_size = imgui.InputInt(
        "Thread block size", thread_block_size)
    changed = imgui.BeginCombo(
        "Initialization strategy", str(initialization_strategy).split('.')[-1])
    if changed:
        for i in range(len(initialization_strategies)):
            _, selected = imgui.Selectable(
                str(initialization_strategies[i]).split('.')[-1], initialization_strategy == initialization_strategies[i])
            if selected:
                initialization_strategy = initialization_strategies[i]
        imgui.EndCombo()
    vbd.initialization_strategy = initialization_strategy
    vbd.kD = kD
    vbd.RdetH = RdetH
    changed, animate = imgui.Checkbox("Animate", animate)
    changed, export = imgui.Checkbox("Export", export)
    step = imgui.Button("Step")
    reset = imgui.Button("Reset")

    if reset:
        vbd.x = mesh.X
        vbd.v = np.zeros(mesh.X.shape)
        vm.update_vertex_positions(mesh.X.T)
        t = 0

    vbd.set_gpu_block_size(thread_block_size)

    if animate or step:
        profiler.begin_frame("Physics")
        vbd.step(dt, iterations, substeps, rho_chebyshev)
        profiler.end_frame("Physics")

        V = vbd.x.T
        if export:
            ps.screenshot(f"{output_path}/{t}.png")

        vm.update_vertex_positions(V)
        t = t+1

    imgui.Text(f"Frame={t}")

# Set up polyscope callback

In [ ]:
ps.set_user_callback(callback)
ps.show()